In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv("../.env-feast")

True

In [2]:
import psycopg2
import os

def query_postgres(query: str):
    """
    Connects to the PostgreSQL database in the 'feast' namespace,
    executes a provided query, and returns the results.

    Args:
        query (str): The SQL query string to be executed.

    Returns:
        list: A list of tuples containing the fetched rows. Returns an empty list on failure.
    """
    conn = None
    rows = []
    try:
        conn_params = {
            'host': 'postgres-postgresql.feast.svc.cluster.local',
            'database': os.environ["POSTGRES_DB"],
            'user': os.environ["POSTGRES_USER"],
            'password': os.environ["POSTGRES_PASSWORD"],
            'port': '5432'
        }

        conn = psycopg2.connect(**conn_params)
        cursor = conn.cursor()

        # Execute the query passed as an argument
        cursor.execute(query)

        # Fetch all results
        if cursor.description:  # Check if the query returns data
            rows = cursor.fetchall()
            print("Query executed successfully.")
        else:
            conn.commit()  # Commit for non-select queries (e.g., INSERT, UPDATE, DELETE)
            print("Non-SELECT query executed successfully.")

    except psycopg2.Error as e:
        print(f"Error executing PostgreSQL query: {e}")

    finally:
        if conn:
            conn.close()
            print("Database connection closed.")
    
    return rows


In [3]:
import pandas as pd

all_entities = query_postgres("SELECT userid, movieid, to_timestamp(timestamp) as timestamp FROM ratings limit 10;")
# entity_df = pd.DataFrame(all_entities, columns=["userid", "movieid", "event_timestamp"])

Error executing PostgreSQL query: relation "ratings" does not exist
LINE 1: ...ovieid, to_timestamp(timestamp) as timestamp FROM ratings li...
                                                             ^

Database connection closed.


In [11]:
import pandas as pd
from datetime import datetime
from feast import FeatureStore

# 1. Instantiate the FeatureStore
store = FeatureStore(repo_path="./feast")

# 2. Define your entity dataframe with historical data.
entity_df_sql = "SELECT userid, movieid, to_timestamp(timestamp) as event_timestamp FROM ratings limit 500000"

# 3. Retrieve the historical features using the FeatureService
training_data = store.get_historical_features(
    entity_df=entity_df_sql,
    features=store.get_feature_service("movie_recommender_service_all"),
)

/opt/conda/envs/movie-recommender-env/lib/python3.11/site-packages/feast/infra/utils/postgres/connection_utils.py:116: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [12]:
# 4. Convert the result to a Pandas DataFrame and print it
df = training_data.to_df()

In [13]:
df

,userid,movieid,event_timestamp,rating,title,genres
0,3788,110,2015-03-13 04:48:00,2.5,Braveheart (1995),Action|Drama|War
1,3788,541,2015-03-13 04:54:02,4.5,Blade Runner (1982),Action|Sci-Fi|Thriller
2,3788,1198,2015-03-13 04:55:16,3.5,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure
3,3788,1291,2015-03-13 04:47:58,3.0,Indiana Jones and the Last Crusade (1989),Action|Adventure
4,3789,1036,2016-04-08 16:33:20,4.5,Die Hard (1988),Action|Crime|Thriller
...,...,...,...,...,...,...
499995,7079,4993,2016-08-07 00:35:49,4.0,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy
499996,7079,5378,2017-08-13 12:46:38,4.5,Star Wars: Episode II - Attack of the Clones (...,Action|Adventure|Sci-Fi|IMAX
499997,7079,6333,2016-08-07 00:37:41,4.0,X2: X-Men United (2003),Action|Adventure|Sci-Fi|Thriller
499998,7079,6764,2016-08-21 00:05:18,4.5,"Rundown, The (2003)",Action|Adventure|Comedy


In [ ]:
import pandas as pd
from datetime import datetime
from feast import FeatureStore

# 1. Instantiate the FeatureStore
store = FeatureStore(repo_path="./feast")

In [19]:
entity_rows = {
        "userid": [1, 1],
        "movieid": [296, 306]
    }

In [20]:
training_data = store.get_online_features(
    entity_rows=entity_rows,
    features=store.get_feature_service("movie_recommender_service_all"),
)

In [21]:
training_data.to_dict()

{'userid': [1, 1],
 'movieid': [296, 306],
 'rating': [5.0, 3.5],
 'genres': ['Comedy|Crime|Drama|Thriller', 'Drama'],
 'title': ['Pulp Fiction (1994)',
  'Three Colors: Red (Trois couleurs: Rouge) (1994)']}

In [26]:
store.get_online_features(
    entity_rows={"movieid": [296]},
    features=[
        "movie_details_feature_view:genres",
        "movie_details_feature_view:title",
    ],
).to_dict()

{'movieid': [296],
 'genres': ['Comedy|Crime|Drama|Thriller'],
 'title': ['Pulp Fiction (1994)']}